In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

from IPython.display import HTML

In [2]:
# importation des données brutes en utilisant les productid comme index

data_path = "../data/raw/"
df = pd.read_csv(data_path+"X_train_update.csv", index_col="productid").drop("Unnamed: 0", axis=1)
y = pd.read_csv(data_path+"Y_train_CVw08PX.csv")['prdtypecode']
y.index = df.index

In [3]:
# on créer une colonne avec le chemin de l'image correspondante

im_folder_path = data_path+"images/image_train/"
# les noms de fichier image sont de la forme image_imageid_product_productid.jpg
df['image'] = im_folder_path + 'image_' + df['imageid'].astype('str') + '_product_' + df.index.astype('str') +'.jpg'
# plus nécessaire de garder imageid qui est renseigné dans le chemin
df.drop('imageid', axis=1, inplace=True)

In [4]:
# fonction qui permet d'afficher les images du dataframe à partir du chemin des images
# et également les descriptions complètent
# attention de ne pas utiliser max_length trop grand sinon ça va être long à charger

def display_df(df, max_length = 100):
    formatters={
        "image": lambda path: f'<img src="{path}" width="400">',
    }
    display(HTML(df.head(max_length).to_html(escape=False, formatters=formatters)))

In [5]:
y.value_counts()[:5]

prdtypecode
2583    10209
1560     5073
1300     5045
2060     4993
2522     4989
Name: count, dtype: int64

In [6]:
# exemple; on affiche les 5 premières entrées de la catégorie 2583

cat = 2583
display_df(df[y == cat], 5)

,designation,description,image
productid,,,
3936362802,Kit de désinfection pour piscines enfants 20990,NaN,
3147314797,Bouée gonflable River tube - OOGARDEN,Bouée gonflable River tube en PVC idéale pour des parties de glissade sur l'eau.Dimensions article gonflé : 107 x 107 x 30 cmDimensions article plat : 122 cmMatière : PVCInclus : Corde de maintienConfortablement installé accrochez vous à cette Bouée gonflable River tube pour des jeux de glisse à la Piscine ou à la mer !,
207151480,Testeur électronique de pH,Utilisation du Testeur électronique de pHGrâce au Testeur électronique de pH la lecture est instantanée la valeur s'affiche sans besoin d'interpréter. De plus le testeur sensibilise à l'importance du pH pour un bon équilibre de l'eau.Caractéristiques techniques du Testeur électronique de pHAffichage numérique Alimentation : 2 piles R9Garantie 1 anUne solution tampon est fournie afin de pouvoir recalibrer l'appareil aprés une dizaine d'utilisation vous garantissant un résultat fiable.,
4128438373,Pompe de filtration Speck Badu 95,NaN,
1857269337,AstralCom 1 CV Mono de DAB - Surpresseur piscine,Tension (V) : Monophasé - 230 V Puissance : 1 CV Turbine (nombre) : 4 Compatible traitement au sel : Oui,


### Tester l'impact de variable indicatrice sur la présence ou non de certains mots

In [8]:
def confusion_matrix(list_of_words, categorie, include_description=True, normalize=False):
    texts = df['designation']          # Series contenant les textes
    if include_description:
        texts += ' - '+df['description'].fillna('')
    contains_list_of_words = np.where(texts.str.lower().str.contains('|'.join(list_of_words)), 1, 0)
    is_in_cat = np.where(y==categorie, 1, 0)
    score = f1_score(is_in_cat, contains_list_of_words)
    cm1 = pd.crosstab(
        is_in_cat,
        contains_list_of_words, rownames=['categorie '+str(categorie)], colnames=['contient un des mots'], normalize=normalize
    )
    cmall = pd.crosstab(y, contains_list_of_words, rownames=['categorie'], colnames=['contient un des mots'], normalize=normalize)
    return cm1, cmall, score

In [10]:
# on voit sur les descriptions de la catégorie 2583 affichés plus haut qu'il y a des pompes, supresseurs, testeurs
# on peut test si il apparaissent dans beaucoup de description de la catégorie 2583 avec les matrice de confusion

cm1, cmall, score = confusion_matrix(["pompe", "supresseur", "testeur"], 2583)
display(cm1)
display(cmall)
print(f"f1 score : {score}")

contient un des mots,0,1
categorie 2583,,
0,74449,258
1,7782,2427


contient un des mots,0,1
categorie,,
10,3116,0
40,2506,2
50,1680,1
60,832,0
1140,2671,0
1160,3953,0
1180,764,0
1280,4869,1
1281,2062,8


f1 score : 0.37645416472778037
